## Marketing Spend / Impression Simulation
This section simulates counterfactual scenarios to estimate the impact of changes in
individual marketing channels on sales while holding other channels constant.


In [1]:
import pandas as pd
import numpy as np
import joblib

df = pd.read_csv("../data/media_spend.csv")
model = joblib.load("../models/final_log_linear_model.pkl")
features = joblib.load("../models/model_features.pkl")


In [2]:
base_input = df[features].mean().to_frame().T
base_input


,Paid_Views,Organic_Views,Google_Impressions,Email_Impressions,Facebook_Impressions,Affiliate_Impressions
0,15093.941658,13355.673222,886173.830875,760509.377761,269126.887906,22910.902655


In [3]:
log_sales_base = model.predict(base_input)[0]
sales_base = np.expm1(log_sales_base)

sales_base


np.float64(128245.23355409087)

In [6]:
def simulate_uplift(
    df,
    model,
    features,
    channel,
    uplift_pct
):
    """
    Simulates the impact of increasing a single marketing channel by uplift_pct
    while holding other channels constant.
    """
    # Baseline
    base = df[features].mean().to_frame().T
    base_log_pred = model.predict(base)[0]
    base_sales = np.expm1(base_log_pred)

    # Scenario
    scenario = base.copy()
    scenario[channel] *= (1 + uplift_pct)

    scenario_log_pred = model.predict(scenario)[0]
    scenario_sales = np.expm1(scenario_log_pred)

    return {
    "Channel": channel,
    "Uplift_%": float(uplift_pct * 100),
    "Base_Sales": float(base_sales),
    "Scenario_Sales": float(scenario_sales),
    "Absolute_Lift": float(scenario_sales - base_sales),
    "Percent_Lift": float(((scenario_sales - base_sales) / base_sales) * 100)
}



In [7]:
user_channel = "Google_Impressions"
user_uplift = 0.10   # 10% increase chosen by user

single_result = simulate_uplift(
    df=df,
    model=model,
    features=features,
    channel=user_channel,
    uplift_pct=user_uplift
)

single_result


{'Channel': 'Google_Impressions',
 'Uplift_%': 10.0,
 'Base_Sales': 128245.23355409087,
 'Scenario_Sales': 129170.35380382178,
 'Absolute_Lift': 925.1202497309132,
 'Percent_Lift': 0.721368135167939}

In [8]:
uplift_pct = 0.10  # user-defined scenario

results = []

for ch in features:
    results.append(
        simulate_uplift(
            df=df,
            model=model,
            features=features,
            channel=ch,
            uplift_pct=uplift_pct
        )
    )

simulation_df = pd.DataFrame(results)
simulation_df.sort_values("Percent_Lift", ascending=False)


,Channel,Uplift_%,Base_Sales,Scenario_Sales,Absolute_Lift,Percent_Lift
3,Email_Impressions,10.0,128245.233554,134240.243681,5995.010127,4.674646
4,Facebook_Impressions,10.0,128245.233554,129659.263602,1414.030048,1.102599
2,Google_Impressions,10.0,128245.233554,129170.353804,925.120250,0.721368
5,Affiliate_Impressions,10.0,128245.233554,129051.402200,806.168646,0.628615
1,Organic_Views,10.0,128245.233554,128592.172730,346.939176,0.270528
0,Paid_Views,10.0,128245.233554,128374.853931,129.620377,0.101072


## Key Business Insights from Simulation

A 10% uplift simulation was conducted independently for each marketing channel
while holding all other channels constant.

Email impressions show the highest marginal impact on sales, suggesting strong
conversion efficiency and potential underinvestment. Paid views show minimal
incremental lift, indicating diminishing returns.

These results can directly guide marketing budget reallocation decisions toward
higher-impact channels.


In [9]:
for pct in [0.05, 0.10, 0.20]:
    print(f"\n--- {int(pct*100)}% Increase Scenario ---")
    display(
        pd.DataFrame([
            simulate_uplift(df, model, features, ch, pct)
            for ch in features
        ]).sort_values("Percent_Lift", ascending=False)
    )



--- 5% Increase Scenario ---


,Channel,Uplift_%,Base_Sales,Scenario_Sales,Absolute_Lift,Percent_Lift
3,Email_Impressions,5.0,128245.233554,131208.503809,2963.270254,2.310628
4,Facebook_Impressions,5.0,128245.233554,128950.310380,705.076825,0.549788
2,Google_Impressions,5.0,128245.233554,128706.962490,461.728936,0.360036
5,Affiliate_Impressions,5.0,128245.233554,128647.686403,402.452849,0.313815
1,Organic_Views,5.0,128245.233554,128418.585981,173.352427,0.135173
0,Paid_Views,5.0,128245.233554,128310.027375,64.793821,0.050523



--- 10% Increase Scenario ---


,Channel,Uplift_%,Base_Sales,Scenario_Sales,Absolute_Lift,Percent_Lift
3,Email_Impressions,10.0,128245.233554,134240.243681,5995.010127,4.674646
4,Facebook_Impressions,10.0,128245.233554,129659.263602,1414.030048,1.102599
2,Google_Impressions,10.0,128245.233554,129170.353804,925.120250,0.721368
5,Affiliate_Impressions,10.0,128245.233554,129051.402200,806.168646,0.628615
1,Organic_Views,10.0,128245.233554,128592.172730,346.939176,0.270528
0,Paid_Views,10.0,128245.233554,128374.853931,129.620377,0.101072



--- 20% Increase Scenario ---


,Channel,Uplift_%,Base_Sales,Scenario_Sales,Absolute_Lift,Percent_Lift
3,Email_Impressions,20.0,128245.233554,140515.497098,12270.263543,9.567813
4,Facebook_Impressions,20.0,128245.233554,131088.884603,2843.651049,2.217354
2,Google_Impressions,20.0,128245.233554,130102.147524,1856.913970,1.447940
5,Affiliate_Impressions,20.0,128245.233554,129862.638502,1617.404948,1.261181
1,Organic_Views,20.0,128245.233554,128940.050467,694.816913,0.541788
0,Paid_Views,20.0,128245.233554,128504.605317,259.371763,0.202247
